# HW5: تولید موسیقی با استفاده از شبکه‌های عصبی بازگشتی
Welcome to Homework 5  &#128522;&#9996;

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<div dir='rtl'style="text-align: right;font-family:verdana;">


ابتدا بسته‌ها و کتابخانه‌های مورد نیاز را import میکنیم، در طول اجرای کد هر پکیج دیگری که نیاز دارید، قرار دهید. 

In [ ]:
!pip install -e /content/drive/MyDrive/deeplearningcourse
import deeplearningcourse as dlc
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Obtaining file:///content/drive/MyDrive/deeplearningcourse
  Attempting uninstall: deeplearningcourse
    Found existing installation: deeplearningcourse 0.0.0
    Can't uninstall 'deeplearningcourse'. No files were found to uninstall.
  Running setup.py develop for deeplearningcourse




## A) مجموعه داده

<div dir='rtl'style="text-align: right;font-family:verdana;">

ابتدا دیتاست مورد نظر را از طریق پکیجی که در گوگل درایو قرار دایم، load کرده و مراحل بعدی را انجام می‌دهیم. 
همانطور که گقتیم دیتاست مورد نظر تعدادی آهنگ محلی ایرلندی است که با نماد ABC نشان داده می‌شود. برای درک بهتر می‌توانیم یکی از این آهنگ‌ها را در نماد ABC نشان دهیم. 
همچنین می‌توانیم این اهنگ را به صورت صوتی نمایش داده و پخش کنیم.





*  یکی از نکات مهمی که باید به آن توجه کنید این است که نماد موسیقی فقط شامل اطلاعاتی در مورد نت‌های در حال پخش نمی‌باشد بلکه علاوه بر این اطلاعات، اطلاعات دیگری مانند عنوان آهنگ، کلید و tempo آهنگ نیز وجود دارد. 



In [ ]:

songs = dlc.P1.load_data()
examplesong = songs[25]
print(examplesong)



Found 817 songs in text
X:24
T:Flowers of Edinburgh
Z: id:dc-hornpipe-23
M:C|
L:1/8
K:G Major
GE|D2DE G2GA|BGBd cBAG|FGEF DEFG|A2FA D2GE|!
D2DE G2GA|BGBd efge|dcBA GFGA|B2G2 G2:|!
(3def|g2gf gbag|f2fe fagf|edef gfed|B2e2 e2ge|!
dBGB d2Bd|edef g2ge|dcBA GFGA|B2G2 G2:|


In [ ]:
joined_songs = "\n\n".join(songs) 
vocab = sorted(set(joined_songs))
print( len(vocab))

83


<div dir='rtl'style="text-align: right;font-family:verdana;">


روند کار به این صورت است که دنباله‌ای از کاراکترها را به مدل وارد می‌کنیم و مدل را برای پیش‌بینی خروجی که کاراکتر بعدی در هر تایم زمانی است، آموزش می‌دهیم. مدل‌های RNN یک intenal state دارند که به عناصر دیده شده قبلی بستگی دارد، بنابراین اطلاعات مربوط به همه کاراکترهای دیده شده تا یک لحظه مشخص برای ایجاد پیش‌بینی گرفته می‌شوند. 





###  A-1) تبدیل متن به بردار

<div dir='rtl'style="text-align: right;font-family:verdana;">



قبل از شروع آموزش مدل RNN باید یک نمایش عددی از مجوعه داده‌های متنی خود ایجاد کنیم. برای این کار می‌توانیم دو جدول مراجعه ایجاد کنیم که یکی از آنها کاراکترها را به عدد نگاشت می‌کند و دیگری اعداد را به کاراکتر برمی‌گراند. فعلا این کار را برای کاراکترهای منحصر به فرد در دیتاست انجام می‌دهیم. 

همچنین می‌توانید این نمایش عددی را برای داده‌های متن مشاهده کنید:

In [ ]:

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:

for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:2d},'.format(repr(char), char2idx[char]))

  '\n':  0,
  ' ' :  1,
  '!' :  2,
  '"' :  3,
  '#' :  4,
  "'" :  5,
  '(' :  6,
  ')' :  7,
  ',' :  8,
  '-' :  9,
  '.' : 10,
  '/' : 11,
  '0' : 12,
  '1' : 13,
  '2' : 14,
  '3' : 15,
  '4' : 16,
  '5' : 17,
  '6' : 18,
  '7' : 19,


<div dir='rtl'style="text-align: right;font-family:verdana;">

**تمرین:**

تابعی بنویسید که همه songs string ها را به نمایش عددی تبدیل کند. توجه داشته باشید که خروجی این تابع باید یک np.array با N عنصر باشد که N تعداد کاراکترهای رشته ورودی است. خروجی تابع را vectorized_songs نامگذاری کنید و در مراحل بعدی استفاده کنید.

In [ ]:
 def vectorize(string):


  # ...


In [ ]:
vectorized_songs = vectorize(joined_songs)

<div dir='rtl'style="text-align: right;font-family:verdana;">


همچنین می‌توان نحوه نگاشت اولین قسمت متن را به یک نمایش integer نمایش داد.

In [ ]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(joined_songs[:20]), vectorized_songs[:20]))



## B)  ایجاد نمونه‌های آموزش و لیبل‌ها

<div dir='rtl'style="text-align: right;font-family:verdana;">

گام بعدی این است که متن را به دنباله‌های نمونه‌ای که در طول آموزش از آن استفاده می‌کنیم، تبدیل کنیم. هر دنباله ورودی که به مدل RNN می‌دهیم دارای یک seq_length است، همچنین باید یک دنباله target برای هر ورودی تعریف کنیم که در آموزش RNN برای پیش‌بینی کاراکتر بعدی استفاده می‌شود. 

مثلا اگر فرض کنیم seq_length برابر با چهار باشد و متن ورودی ما "Hello"  باشد، دنباله هدف "ello" است. 



*   روش دسته‌ای به ما این امکان را می‌دهد که این جریان کاراکترها را به دنباله‌هایی با اندازه‌های دلخواه تبدیل کنیم.

* 
تابع creat_examples نمونه‌های مورد نظر ما را ایجاد می‌کند و از آن برای تولید داده‌های آموزش و لیبل‌های آن‌ها استفاده می‌کنیم. 

In [ ]:
#  creat training examples and targets 

def creat_examples(vectorized_songs, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]


  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch

<div dir='rtl'style="text-align: right;font-family:verdana;">


   برای درک بهتر می‌توانید چند کاراکتر اول در متن را به همراه target آن مشاهده کنید.




In [ ]:

x_train, y_train = creat_examples(vectorized_songs, seq_length= 5, batch_size= 1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))



## C) شبکه‌های عصبی بازگشتی

### C-1) تعریف مدل

<div dir='rtl'style="text-align: right;font-family:verdana;">

پس از آماده‌سازی داده‌های آموزش و تست، می‌خواهیم یک شبکه عصبی بازگشتی طراحی کنیم، آموزش دهیم و سپس از مدل آموزش دیده برای تولید موسیقی جدید استفاده کنیم. 

**تمرین:**

مدل پیشنهادی خود را ارائه کنید و آن را آموزش دهید:

In [ ]:
# Defining the RNN Model 
# TODO

In [ ]:
model.summary()

<div dir='rtl'style="text-align: right;font-family:verdana;">

**تمرین:**

با در نظر گرفتن طول ورودی برابر با 100 و استفاده از تابع pred اندازه خروجی را نشان دهید:
 

*   
توجه داشته باشید که می‌توان مدل را بر روی ورودی‌هایی با هر اندازه آموزش داد.









In [ ]:
x, y = creat_examples(vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)

# TODO

<div dir='rtl'style="text-align: right;font-family:verdana;">

همچنین می‌توانیم رمزگشایی انجام دهیم تا ورودی را با متن پیش‌بینی شده توسط مدل آموزش ندیده مقایسه کنیم:

In [ ]:
sampled_indices = tf.random.categorical(pred[10], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

In [ ]:
print("input: \n", repr("".join(idx2char[x[10]])))
print("char prediction: \n", repr("".join(idx2char[sampled_indices])))

### C-2) آموزش مدل

<div dir='rtl'style="text-align: right;font-family:verdana;">



*   همانطور که مشاهده می‌کنیم متن تولید شده توسط مدل آموزش ندیده بسیار بی معنی است و باید برای بهبود مدل را آموزش دهیم. 





**تمرین:**

تابع هزینه مناسب را برای محاسبه و برگرداندن خطا بین لیبل‌های واقعی و پیش‌بینی شده تعریف کنید. 
 ( از تابع هزینه sparse_categorical_crossentropy  استفاده کنید.)

In [ ]:
# Defining the loss function
def loss_function(labels, logits):
  # TODO
  return loss

<div dir='rtl'style="text-align: right;font-family:verdana;">


**تمرین:**

تابع خطا را برای محاسبه و برگرداندن مقدار خطا بین مقدار واقعی و مقدار پیش‌بینی شده مدل آموزش ندیده اعمال کنید و مقدار خطا را نشان دهید:

In [ ]:
# TODO

<div dir='rtl'style="text-align: right;font-family:verdana;">


**تمرین:**

با تعریف هایپرپارامترها به صورت مناسب مدل را آموش دهید:



-    هایپرپارامترهای مناسب را برای شبکه انتخاب کرده و نحوه به دست آوردن این هایپرپارامترها را توضیح دهید.

-  بعد از تعریف هایپرپارامترها مدل را آموزش داده و دقت آن را روی داده‌های آموزش گزارش دهید. همچنین نمودار خطا را در طی فرایند آموزش رسم کنید.
-    تعداد ایپوک‌ها در مرحله آموزش چگونه بر عملکرد مدل تاثیر می‌گذارند؟
    



In [ ]:
### Hyperparameter setting and optimization ###
num_training_iterations =  #TODO
batch_size =   # TODO
seq_length =   # TODO
learning_rate = #TODO


# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [ ]:
### train model




## D) تولید موسیقی با استفاده از شبکه‌های عصبی بازگشتی

<div dir='rtl'style="text-align: right;font-family:verdana;">

اکنون می‌توانیم از RNN آموزش دیده برای تولید موسیقی استفاده کنیم. هنگام تولید موسیقی، ما باید مدل را به نوعی تغذیه کنیم تا شروع به کار کند. هنگامی که یک seed تولید شده داریم، می‌توانیم هر یک از کاراکترهای پی در پی را به صورت تکراری پیش‌بینی کنیم. 




*   به این نکته توجه داشته باشید که مدل RNN از یک لایه softmax  در لایه آخر استفاده می‌کند که برای نتیجه‌گیری به طور مکرر از این توزیع‌ها نمونه می‌گیریم و سپس از نمونه‌های خود برای کدگذاری آهنگ تولید شده قالب ABC استفاده می‌کنیم. 

**تمرین:**

تابع زیر را برای تولید موسیقی با استفاده از مدل آموزش دیده خود تمکیل کنید:


In [ ]:

def text_generation(model, start_string, generation_length=1000):
# TODO

<div dir='rtl'style="text-align: right;font-family:verdana;">


**تمرین:**

اکنون می‌توانید موسیقی‌های تولید شده (text)
را به صورت زیر یا به هر طریقی که می‌توانید، در قالب ABC نشان داده یا پخش کنید.

In [ ]:
songs = dlc.P1.extract_songsnippet(text)
for i, song in enumerate(songs): 
  dlc.P1.play(song)
  print(song, end="\n\n\n\n")